In this competition, you will predict sales for the thousands of product families sold at Favorita stores located in Ecuador. The training data includes dates, store and product information, whether that item was being promoted, as well as the sales numbers. Additional files include supplementary information that may be useful in building your models.

Import

In [ ]:
import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib
import matplotlib.style
import matplotlib as mpl
mpl.style.use('classic')
from mpl_toolkits.mplot3d import Axes3D

Load

In [ ]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

Read files

In [ ]:
train = pd.read_csv("/kaggle/input/store-sales-time-series-forecasting/train.csv")
train

In [ ]:
train.info()

In [ ]:
test = pd.read_csv("/kaggle/input/store-sales-time-series-forecasting/test.csv")
test

In [ ]:
oil = pd.read_csv("/kaggle/input/store-sales-time-series-forecasting/oil.csv")
oil

holiday2

In [ ]:
holiday = pd.read_csv("/kaggle/input/store-sales-time-series-forecasting/holidays_events.csv")
holiday

In [ ]:
type_hols = holiday['locale'].value_counts()
type_hols

Map type_hols

In [ ]:
dict_hols = {"National":1, "Regional":2, "Local":3}
holiday['type_hols'] = holiday['locale'].map(dict_hols)
holiday

In [ ]:
holiday.info()

In [ ]:
holiday2 = pd.DataFrame()

holiday2['date'] = oil['date']
holiday2['type_hols'] = 0
holiday2

In [ ]:
type_holiday = []

i = 0
j = 0

for i in range(len(holiday2)):
    for j in range(len(holiday)):
        if holiday2['date'].iloc[i] == holiday['date'].iloc[j]:
            num_holiday = holiday['type_hols'].iloc[j]
            j = j + 1
            break
        else:
            num_holiday = 0
            j = j + 1
    type_holiday.append(num_holiday)
    i = i + 1
print(type_holiday)
print(len(type_holiday))

In [ ]:
holiday2['type_hols'] = type_holiday
holiday2

In [ ]:
stores = pd.read_csv("/kaggle/input/store-sales-time-series-forecasting/stores.csv")
stores

In [ ]:
transactions = pd.read_csv("/kaggle/input/store-sales-time-series-forecasting/transactions.csv")
transactions

In [ ]:
submission = pd.read_csv("/kaggle/input/store-sales-time-series-forecasting/sample_submission.csv")
submission

Analyse target

In [ ]:
sns.distplot(train['sales'])

Combine train and test

In [ ]:
target = train['sales']

train_copy = train

combi = train_copy.drop(['sales'], axis=1).append(test)
combi

Map oil to combi

In [ ]:
combi = combi.assign(dcoilwtico=combi.date.map(oil.set_index('date').dcoilwtico))
combi

Map holiday to combi

In [ ]:
combi = combi.assign(type_hols=combi.date.map(holiday2.set_index('date').type_hols))
combi

Combine holiday and combi

In [ ]:
combi.info()

Create new column

In [ ]:
#combi.insert(6,'type_hols', 0)
#combi

In [ ]:
#type_holiday = []

#i = 0
#j = 0

#for i in range(len(combi)):
#    for j in range(len(holiday)):
#        if combi['date'].iloc[i] == holiday['date'].iloc[j]:
#            num_holiday = holiday['type_hols'].iloc[j]
#            j = j + 1
#            break
#        else:
#           num_holiday = 0
#           j = j + 1
#   type_holiday.append(num_holiday)
#   i = i + 1
#print(type_holiday)
#print(len(type_holiday))

Search for null values

In [ ]:
combi.isnull().sum()

Replace nan with 0

In [ ]:
combi['dcoilwtico'] = combi['dcoilwtico'].fillna(0)
combi['type_hols'] = combi['type_hols'].fillna(0)
combi

Ordinal encode categories

In [ ]:
from sklearn import preprocessing
from sklearn.preprocessing import OrdinalEncoder

enc = OrdinalEncoder()

for col in combi:
    if combi[col].dtype=="object":
        combi[col] = enc.fit_transform(combi[col].values.reshape(-1,1))
combi

Heatmap

In [ ]:
corrmat = combi.corr()
f, ax = plt.subplots(figsize=(12, 9))
sns.heatmap(corrmat, vmax=.8, square=True);

Normalise

In [ ]:
combi = (combi - combi.min()) / (combi.max() - combi.min())
combi

Standardise

In [ ]:
#combi = (combi = combi.min()) / np.std(combi)
#combi

Define X, y and X_test

In [ ]:
features = ['date', 'store_nbr', 'family', 'onpromotion', 'dcoilwtico']

y = target
X = combi[: len(train)][features]
X_test = combi[len(train) :][features]

Split

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.1, random_state=42)
X_train.shape, X_val.shape, y_train.shape,y_val.shape, X_test.shape

Select model

In [ ]:
from sklearn.linear_model import LinearRegression

model = LinearRegression().fit(X_train, y_train)
print(model.score(X_train, y_train))

Predict on validation set

In [ ]:
y_pred = model.predict(X_val)
print(model.score(X_val, y_val))

In [ ]:
from sklearn import metrics
print('Mean Absolute Error:', metrics.mean_absolute_error(y_val, y_pred))
print('Mean Squared Error:', metrics.mean_squared_error(y_val, y_pred))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_val, y_pred)))

In [ ]:
fig, ax = plt.subplots()
ax.scatter(y_val, y_pred, edgecolors=(0, 0, 0))
ax.plot([y.min(), y.max()], [y.min(), y.max()], 'k--', lw=4)
ax.set_xlabel('Measured')
ax.set_ylabel('Predicted')
plt.show()

Predict on test set

In [ ]:
prediction = model.predict(X_test)
prediction[prediction < 0] = 0
submission['sales'] = prediction

In [ ]:
submission.to_csv('submission.csv', index=False)
submission = pd.read_csv("submission.csv")
submission